<a href="https://colab.research.google.com/github/klubgllrt/RoughWaterway_GeoTools/blob/master/Water_finder_osmnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip install  osmnx

In [0]:
!pip install reverse-geocode

In [0]:
!sudo apt-get install libgeos-3.5.0
!sudo apt-get install libgeos-dev
!sudo pip install https://github.com/matplotlib/basemap/archive/master.zip

In [0]:
!sudo apt install libspatialindex-dev

In [0]:
! pip install overpy

In [0]:
! pip install fuzzywuzzy

In [0]:
import geopandas as gpd #
import pandas as pd #
import shapely
from shapely.geometry import shape
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points
from json import loads
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import osmnx as ox
ox.config(log_file=True, log_console=True, use_cache=True)
import numpy as np 

crs = {'init': 'epsg:4326'}
pd.set_option('display.max_columns', None)

In [0]:
from fuzzywuzzy import fuzz

In [0]:
df = pd.read_excel('WW Freshwater Datashare.xlsx')
df.head()

In [0]:
df.rename(columns={
                    'Sample Lat/Long - Latitude':'Latitude',
                    'Sample Lat/Long - Longitude':'Longitude',
                  }, 
                 inplace=True)
df = df.dropna(subset=['Latitude', 'Longitude'])

In [0]:
#gdf_as = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

# creating a geometry column 
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]

# Creating a Geographic data frame 
gdf_as = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)

gdf_as.head()

In [0]:
def make_plot(G, footprints=False):
    fig, ax = ox.plot.plot_graph(G, 
                               bbox=None, 
                               fig_height=6, 
                               fig_width=None, 
                               margin=0.02, 
                               axis_off=True, 
                               equal_aspect=False, 
                               bgcolor='w', 
                               show=True, 
                               save=False, 
                               close=True, 
                               file_format='png', 
                               filename='temp', 
                               dpi=300, 
                               annotate=False, 
                               node_color='#66ccff', 
                               node_size=15, 
                               node_alpha=1, 
                               node_edgecolor='none', 
                               node_zorder=1, 
                               edge_color='#999999', 
                               edge_linewidth=1, 
                               edge_alpha=1, 
                               use_geom=True)

In [0]:
def get_network_graph(point, dist=1000):
  G = ox.core.graph_from_point(point, 
                              distance=dist, 
                              distance_type='network', 
                              network_type='all', 
                              simplify=True, 
                              retain_all=False, 
                              truncate_by_edge=False, 
                              name='unnamed', 
                              timeout=20, 
                              memory=None, 
                              max_query_area_size=2500000000, 
                              clean_periphery=True, 
                              infrastructure='way["waterway"]', 
                              custom_filter=None)
  return G

In [0]:
def get_gdf_from_G(G):
  gdf = ox.save_load.graph_to_gdfs(G, 
                                   nodes=False, 
                                   edges=True, 
                                   node_geometry=False, 
                                   fill_edge_geometry=True)
  return gdf

In [0]:
def get_water_fp_place(place):
  gdf_fp=ox.footprints.footprints_from_place(place, footprint_type='waterway', retain_invalid=False)
  # gdf_fp[gdf_fp.waterway=='drain'].plot()
  return gdf_fp
  

In [0]:
from shapely.ops import cascaded_union

def comb_poly(gdf_s, buff_size =.1):
  gdf_s['geometry']= gdf_s.geometry.buffer(buff_size)
  polygons = gdf_s['geometry'].to_list()
  boundary = gpd.GeoSeries(cascaded_union(polygons))
  poly = boundary[0]
  return poly
def get_water_from_gdf(gdf, buf=.1, rend=0):
  poly1 = comb_poly(gdf, buf)
  gdf_f = ox.footprints.footprints_from_polygon(poly1, footprint_type='waterway', retain_invalid=False)
  return gdf_f

# point= (45.124858, -92.746426)
# G = get_network_graph(point, dist=1000)
# gdf = get_gdf_from_G(G)

# pol = get_water_from_gdf(gdf)
# pol.plot()

  
  

# gdf_f 
# # gdf_s.plot()
#df2 = ox.footprints.create_footprints_gdf(gdf.iloc[0], north=None, south=None, east=None, west=None, footprint_type='building', retain_invalid=False)


In [0]:
!pip install reverse_geocoder

In [0]:
import reverse_geocoder as rg
def lookupgeo(indx=1):
  i= indx
  lat= gdf_as.geometry.y.iloc[i]
  lon= gdf_as.geometry.x.iloc[i]
  results = rg.search([lat, lon])
  for idx, city in enumerate(results):
      gdata = results[idx]
      QRYcity=gdata['name']
      QRYadmin1= gdata['admin1']
      QRYadmin2= gdata['admin2']
      QRYcc=gdata['cc']
      return QRYcity,QRYadmin1, QRYadmin2, QRYcc 

lookupgeo()

In [0]:
import reverse_geocoder as rg
def lookupgeo2(gdf):
  i= indx
  lat= gdf.geometry.y
  lon= gdf.geometry.x
  print(lat, lon)
  results = [lat, lon].apply(lambda lat, lon: rg.search("[", lat,"," lon"]")
  for idx, city in enumerate(results):
      gdata = results[idx]
      QRYcity=gdata['name']
      QRYadmin1= gdata['admin1']
      QRYadmin2= gdata['admin2']
      QRYcc=gdata['cc']
      return QRYcity,QRYadmin1, QRYadmin2, QRYcc 
    
gdf_as.shape

In [0]:
r=range(0, 2401)
gdf_as['QRYcity'] = 'e'
gdf_as['QRYadmin1'] = 'e'
gdf_as['QRYadmin2']= 'e'
gdf_as['QRYcc']='e'

for i in r: 
  QRYcity,QRYadmin1, QRYadmin2, QRYcc = lookupgeo(i)
  gdf_as['QRYcity'][i] = QRYcity
  gdf_as['QRYadmin1'][i] = QRYadmin1
  gdf_as['QRYadmin2'][i] = QRYadmin2
  gdf_as['QRYcc'][i] = QRYcc
  print(i)

gdf_as.head()  

In [0]:
gdf_as

In [0]:
def do_ox_analysis(gdf, index=1, Loud=False, Lean = False):
  gdf_a = gdf
  i = index
  ogloc = gdf_a[['Sample Location']].iloc[i]
  print('index: ', i)
  
  QRYcity,QRYadmin1, QRYadmin2, QRYcc = lookupgeo(i)  
  
 
  
  msg =''
  point = (gdf_a.geometry.y[i], gdf_a.geometry.x[i])
  place = gdf_a['Sample Location'][i]
  
  
  try:
    Method='Network'
    G = get_network_graph(point, dist=1000)
    gdf = get_gdf_from_G(G)
    most_common_name = gdf['name'].value_counts().idxmax()
    gdf['new_name']=most_common_name
    gdf = get_water_from_gdf(gdf)
#     msg = "G-> gdf: GOOD"
#     print('\t', msg)
#     make_plot(G) 
  except: 
    msg = "WARNING!!!! G-> gdf: FAILED"
    #print('\t', msg)
    #print("\t Could not generate network graph... seaching location")
  
    try:
      Method='Location'
      gdf = get_water_fp_place(place)
#       gdf.plot()
#       plt.show()
    except:  
      msg = "WARNING!!!! Location: FAILED"
      #print('\t', msg)
 
      try:
        Method='Geocoded Location'
        gdf = get_water_fp_place(QRYcity)
#         gdf.plot()
#         plt.show()
      except:  
        Method='Fail'
        print('Method: FAILURE',msg, "NO RESULTS :(", "\n\n")
        return
  keep_list = ['Record', 'Total Other Shape','Total Filament','geometry','Continent', 'Country', 'Depth of Water Body (approximate, FW only)', 'Exposed rocks present in sampling area? (FW only)', 'Notes', 'Number of Filters', 'Sample Date', 'Latitude', 'Longitude', 'Sample Location', 'Sample Time (local)', 'Sample Volume (L)', 'Sample Year', 'Sample location (Urban, Rural, Remote)', 'Sample waters', 'Sampling Platform', 'Sediment Load', 'Lab Notes', 'Plastic Type', 'Total Pieces', 'Total Pieces/L']
  gdf_a = gdf_a[keep_list]
  gdf['Original_name'] = ogloc
  if Lean == False:
    gdf['QRYcity'] = QRYcity
    gdf['QRYadmin1'] = QRYadmin1
    gdf['QRYadmin2'] = QRYadmin2
    gdf['QRYcc'] = QRYcc
    gdf['loop'] = i
    gdf['MapMethod'] = Method
    gdf['Record']= gdf_a['Record'].iloc[i]
    gdf['Sample Date']= gdf_a['Sample Date'].iloc[i]
    gdf['Year']= gdf_a['Sample Year'].iloc[i]
    gdf['Sample Time (local)']= gdf_a['Sample Time (local)'].iloc[i]
    gdf['Latitude']= gdf_a['Latitude'].iloc[i]
    gdf['Longitude']= gdf_a['Longitude'].iloc[i]
    gdf['Continent']= gdf_a['Continent'].iloc[i]
    gdf['Country']= gdf_a['Country'].iloc[i]
    gdf['Sample_waters']= gdf_a['Sample waters'].iloc[i]
    gdf['Sample_location_class']= gdf_a['Sample location (Urban, Rural, Remote)'].iloc[i]
    gdf['Notes']= gdf_a['Notes'].iloc[i]
    gdf['Total_Filament']= gdf_a['Total Filament'].iloc[i]	
    gdf['Total_Other_Shape'] = gdf_a['Total Other Shape'].iloc[i]	
    gdf['Total_Pieces']= gdf_a['Total Pieces'].iloc[i]
    gdf['PiecesPerLiter']= gdf_a['Total Pieces/L'].iloc[i]	
    gdf['Plastic_Type']= gdf_a['Plastic Type'].iloc[i]
    gdf['Lab_Notes']= gdf_a['Lab Notes'].iloc[i]

  print('Method: ',Method, '\n\n')
  #if Loud ==True:

     # print(gdf_a[['Sample Location','Continent', 'Country', 	'Sample waters', 	'Sample location (Urban, Rural, Remote)']].iloc[i])






  return gdf

In [0]:

err = 0 
gdfm = gpd.GeoDataFrame()
debug=False


r=range(0, 1008)

if debug==True:
  print(gdfm)
  r = range(15, 20)
  
for i in r: 
  
  gdf= do_ox_analysis(gdf_as, index=i, Loud=debug)
  try:
    gdfm = gdfm.append(gdf)
#     gdf.plot() 
  except:
    err+=1

# if debug==True:
print('ERROR_RATE: ',((err/(i+1))*100),'%')


gdfm.head()  

In [0]:
df= gdfm

# df['Dt'] = pd.to_datetime((gdfm['Sample Date'].astype(str)+ ' ' + gdfm['Sample Time (local)'].astype(str)), errors='coerce')
df = df.drop(['Sample Date', 'Sample Time (local)', 'nodes'], axis=1)
#df = df[~df[geometry]].astype(str, errors='ignore') 
# df =df.reset_index()
# df['Dt'] = df['Dt'].astype(str)
print(df.info())
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs=crs)
gdf.head()

from google.colab import files
gdf.to_file("MPC_MAP_FreshWater.shp")
files.download('MPC_MAP_FreshWater.shp')

In [0]:

print(gdfm['MapMethod'].value_counts())
# temp = gdfm[gdfm['MapMethod']=='Location']
# temp[temp['loop']==1].plot()

In [0]:
#gdfw = gdfm.select_dtypes(exclude=['datetime64'])
gdf_as

In [0]:
dfw.to_csv('WWMP_Poly_out.csv')

In [0]:
dfw.group

In [0]:
# d1 = {'geo': ['start'], 'loct': ['start']}
gfd_l = gpd.GeoDataFrame()
gfd_l.reset_index()
gfd_l

In [0]:
dfg = pd.read_csv('WWMP_Poly_out.csv')

In [0]:
import pandas as pd

In [0]:
dfc = pd.read_csv('WWMP_small.csv')
dfc.columns.to_list()

In [0]:
!pip install geopandas

In [0]:
import geopandas as gpd

In [0]:
geometry = [Point(xy) for xy in zip(dfc['Longitude'], dfc['Latitude'])]

# Creating a Geographic data frame 
gdf_as = gpd.GeoDataFrame(dfc, crs=crs, geometry=geometry)

gdf_as.head()

In [0]:
gdf_as.plot(color='blue')


In [0]:
gdf_as.to_file('WW_MP_with_info.shp')

In [0]:
gdfcsv = gpd.read_file('WW_MP.shp')
gdfcsv.info()

In [0]:
dfw=pd.DataFrame(gdfcsv)
dfw.info()

In [0]:
dfw.to_csv('geocoded_WWMP_small.csv')

In [0]:
gdf = gdf_as
gdf['geometry']= gdf.geometry.buffer(1)
gdf.plot(color='red')
plt.show()

In [0]:
gdf.to_file("1_Val_WW_MP_BufferedPoints.shp")

In [0]:
dist =1000
pt = (38.33486, -81.61276)
sname = 'Kanawha River'

map_settings = dict(
      dist=10000,
      edge_color='k',
      bgcolor='w',
      dpi = 300,
      point = (38.8072489,-77.0479908),
      default_width=2,
      )
print('gotdist: ',dist)
map_settings['dist'] = dist
print('gotpoint: ',pt)
map_settings['point'] = pt #frame['Latitude'], frame['Longitude']#.geometry.y, frame.geometry.x


#   fig, ax = ox.plot_figure_ground(network_type='all', 
#     **map_settings,
#     )

##### get info from row
print("2 !!!!!gothere")
#sname = frame['Sample Location']



##### find water & store to frame
print("3 !!!!!gothere")
gdf_water = ox.footprints.footprints_from_point(point=map_settings['point'], 
    distance=map_settings['dist'], footprint_type='waterway',
    )

print(gdf_water)


##### check title
looks = pd.DataFrame()
wname= []
pRat=[]

for name in gdf_water.name: 
  Str1 = str(name)
  Str2 = str(sname)
  if Str2.lower() != 'nan':
    Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
    Partial_Ratio = fuzz.partial_ratio(Str1.lower(),Str2.lower())
    wname.append(Str1)
    pRat.append(Partial_Ratio)
    looks = pd.DataFrame({'name':name,
                          'pRat':pRat})

    ##### did title work?
if looks[looks.pRat>60].name.count() >0:

  lookmx = looks.loc[looks['pRat'].idxmax()]
  name = lookmx[0]
  gdf_l = ox.core.gdf_from_place(name, gdf_name=name, which_result=1, buffer_dist=None) #     '[waterway=*]' +

  print('MATCH!!! \n\n')

else:
  print('no matches')



#print(gdf_water.info())
gdf_l.plot()
print(gdf_l)
gdf_water.plot()

#   if plotwater==True:

#     north, south, east, west = ox.bbox_from_point(map_settings['point'], map_settings['dist'])
#     G = ox.graph_from_bbox(north=north, south=south, east=east, west=west,
#                            retain_all=True, truncate_by_edge=True, simplify=True,
#                            network_type='none', infrastructure='way["waterway"]')
#     fig, ax = ox.plot_graph(ox.project_graph(G))

#     return gdf_water, fig, gdf_l 




In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import osmnx as ox
ox.config(log_file=True, log_console=True, use_cache=True)
import numpy as np 

def getwater(frame, dist=1000, plotwater=False):
  print(frame['Sample Location'])
  pt = (frame['Latitude'], frame['Longitude'])
  
  try: 
    ###### ox chart settings

    map_settings = dict(
        dist=10000,
        edge_color='k',
        bgcolor='w',
        dpi = 300,
        point = (38.8072489,-77.0479908),
        default_width=2,
        )
    print('gotdist: ',dist)
    map_settings['dist'] = dist
    print('gotpoint: ',pt)
    map_settings['point'] = pt #frame['Latitude'], frame['Longitude']#.geometry.y, frame.geometry.x


  #   fig, ax = ox.plot_figure_ground(network_type='all', 
  #     **map_settings,
  #     )

    ##### get info from row
    sname = frame['Sample Location']



    ##### find water & store to frame
    gdf_water = ox.footprints.footprints_from_point(point=map_settings['point'], 
        distance=map_settings['dist'], footprint_type='waterway',
        )

    print(gdf_water)


    ##### check title
    looks = pd.DataFrame()
    wname= []
    pRat=[]

    for name in gdf_water.name: 
      Str1 = str(name)
      Str2 = str(sname)
      if Str2.lower() != 'nan':
        Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
        Partial_Ratio = fuzz.partial_ratio(Str1.lower(),Str2.lower())
        wname.append(Str1)
        pRat.append(Partial_Ratio)
        looks = pd.DataFrame({'name':name,
                              'pRat':pRat})

        ##### did title work?
    if looks[looks.pRat>60].name.count() >0:

      lookmx = looks.loc[looks['pRat'].idxmax()]
      name = lookmx[0]
      gdf_l = ox.core.gdf_from_place(name, gdf_name=name, which_result=1, buffer_dist=None) #     '[waterway=*]' +

      print('MATCH!!! \n\n')

    else:
      print('no matches')



    #print(gdf_water.info())
    gdf_l.plot()
    print(gdf_l)
    gdf_water.plot()
    return gdf_l
    if plotwater==True:

      north, south, east, west = ox.bbox_from_point(map_settings['point'], map_settings['dist'])
      G = ox.graph_from_bbox(north=north, south=south, east=east, west=west,
                             retain_all=True, truncate_by_edge=True, simplify=True,
                             network_type='none', infrastructure='way["waterway"]')
      fig, ax = ox.plot_graph(ox.project_graph(G))

      return gdf_water, fig, gdf_l 
  
  except:
    print("An exception occurred at: ", sname)
    gdf_l = 'oops'
  return gdf_l

In [0]:
osmnx.footprints.footprints_from_point()

In [0]:
gdf_t = pd.DataFrame(gdf_as.iloc[[0]])

#print(gdf_t.geometry)
gdf_try = gdf_t.apply(getwater, axis=1, args=(1000, True))
gdf_try

In [0]:
import overpy
api = overpy.Overpass()
r = api.query("""
area["ISO3166-1"="DE"][admin_level=2];
(node["amenity"="biergarten"](area);
 way["amenity"="biergarten"](area);
 rel["amenity"="biergarten"](area);
);
out center;
""")
coords  = []
coords += [(float(node.lon), float(node.lat)) 
           for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
           for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
           for rel in r.relations]

In [0]:
# Convert coordinates into numpy array
X = np.array(coords)
plt.plot(X[:, 0], X[:, 1], 'o')
plt.title('Biergarten in Germany')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.axis('equal')
plt.show()

In [0]:
import overpy

api = overpy.Overpass()

# fetch all ways and nodes
result = api.query("""
    way(50.746,7.154,50.748,7.157) ["highway"];
    (._;>;);
    out body;
    """)

for way in result.ways:
    print("Name: %s" % way.tags.get("name", "n/a"))
    print("  Highway: %s" % way.tags.get("highway", "n/a"))
    print("  Nodes:")
    for node in way.nodes:
        print("    Lat: %f, Lon: %f" % (node.lat, node.lon))

In [0]:
i=17
point = (gdf_as.geometry.y[i], gdf_as.geometry.x[i])
place = gdf_as['Sample Location'][i]
distance = 100000
print("Name: ", place)
print("point: ", point)

#gdf = ox.footprints.footprints_from_point(point, distance, footprint_type='waterway', retain_invalid=False)
gdf = ox.footprints.footprints_from_place(place, footprint_type='waterway', retain_invalid=False)
gdf.plot()
gdf.head()

In [0]:



#gdf.groupby('name').count()
gdf

In [0]:
from fuzzywuzzy import fuzz

In [0]:


dfgrowing = gpd.GeoDataFrame()
for i in range(0,1000):
  try:
    point =  gdf_as.geometry[i].y, gdf_as.geometry[i].x
    print(point)
    #df, fig, gdf_l = getwater(gdf_as['Sample Location'][i], point, distance= 1000)
    df, fig, gdf_l = getwater(gdf_as['Sample Location'][i], point, distance= 1000, plotwater=True)

    print('Sample Location: ', gdf_as['Sample Location'][i])
    print('Sample Waters: ',gdf_as['Sample waters'][i])


    gdf_t = pd.DataFrame(gdf_as.iloc[[i]])
    gdf_t = gdf_t.reset_index()
    del gdf_t['geometry']
    gdf_t['bbox_east']=gdf_l['bbox_east']
    gdf_t['bbox_north']=gdf_l['bbox_north']
    gdf_t['bbox_west']=gdf_l['bbox_west']
    gdf_t['geometry']=gdf_l['geometry']
    gdf_t['place_name']=gdf_l['place_name']
    gdf_t.head()
    dfgrowing.append(gdf_t)
  except:
    print("An exception occurred at: ", i)
   
    gdf_t = pd.DataFrame(gdf_as.iloc[[i]])
    gdf_t = gdf_t.reset_index()
    gdf_t['bbox_east']= '9999'
    gdf_t['bbox_north']='9999'
    gdf_t['bbox_west']='9999'
    gdf_t['geometry']='9999'
    gdf_t['place_name']='9999'
    dfgrowing.append(gdf_t)

dfgrowing.head()

In [0]:

gdf_l = ox.core.gdf_from_place('Hells Canyon Tank, AZ', gdf_name='Mississippi River, Minnesota', which_result=1, buffer_dist=None)
gdf_l.columns.tolist()

In [0]:
%matplotlib inline
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from geopandas import GeoDataFrame
from mpl_toolkits.basemap import Basemap
from shapely.geometry import Point

In [0]:
# define basemap colors
land_color = '#F6F6F6'
water_color = '#D2F5FF'
coastline_color = '#333333'
border_color = '#999999'


# first define a transverse mercator projection
map_width_m = 1000 * 1000
map_height_m = 1200 * 1000
target_crs = {'datum':'WGS84',
              'ellps':'WGS84',
              'proj':'tmerc',
              'lon_0':-119,
              'lat_0':37.5}


# plot the map
fig_width = 6
plt.figure(figsize=[fig_width, fig_width * map_height_m / float(map_width_m)])

m = Basemap(ellps=target_crs['ellps'],
            projection=target_crs['proj'],
            lon_0=target_crs['lon_0'], 
            lat_0=target_crs['lat_0'],
            width=map_width_m, 
            height=map_height_m,
            resolution='l',
            area_thresh=10000)

m.drawcoastlines(color=coastline_color)
m.drawcountries(color=border_color)
m.fillcontinents(color=land_color, lake_color=water_color)
m.drawstates(color=border_color)
m.drawmapboundary(fill_color=water_color)


plt.show()

In [0]:
import osmnx as ox
ox.config(use_cache=True, log_console=True)
point = (44.97701667, -93.24093333)
dist = 10000
north, south, east, west = ox.bbox_from_point(point, distance=dist)
G = ox.graph_from_bbox(north=north, south=south, east=east, west=west,
                       retain_all=True, truncate_by_edge=True, simplify=False,
                       network_type='none', infrastructure='way["waterway"]')
fig, ax = ox.plot_graph(ox.project_graph(G))

In [0]:
ox.save_load.graph_to_gdfs(G, nodes=True, edges=True, node_geometry=True, fill_edge_geometry=True)

In [0]:
point = (44.97701667, -93.24093333)
ox.pois.pois_from_point(point, distance=None, amenities=None)

In [0]:
# get the place shape
gdf = ox.gdf_from_place('Portland, Maine')
gdf = ox.project_gdf(gdf)

# get the street network, with retain_all=True to retain all the disconnected islands' networks
G = ox.graph_from_place('Portland, Maine', network_type='drive', retain_all=True)
G = ox.project_graph(G)

In [0]:
# plot the network, but do not show it or close it yet
fig, ax = ox.plot_graph(G, fig_height=10, show=False, close=False, edge_color='#777777')
plt.close()

In [0]:
from descartes import PolygonPatch
from shapely.geometry import Polygon, MultiPolygon

In [0]:
# to this matplotlib axis, add the place shape as descartes polygon patches
for geometry in gdf['geometry'].tolist():
    if isinstance(geometry, (Polygon, MultiPolygon)):
        if isinstance(geometry, Polygon):
            geometry = MultiPolygon([geometry])
        for polygon in geometry:
            patch = PolygonPatch(polygon, fc='#cccccc', ec='k', linewidth=3, alpha=0.1, zorder=-1)
            ax.add_patch(patch)

In [0]:
# optionally set up the axes extents all nicely
margin = 0.02
west, south, east, north = gdf.unary_union.bounds
margin_ns = (north - south) * margin
margin_ew = (east - west) * margin
ax.set_ylim((south - margin_ns, north + margin_ns))
ax.set_xlim((west - margin_ew, east + margin_ew))
fig

In [0]:
crs = {'init': 'epsg:4326'}
gdf = gdf.to_crs(crs)

In [0]:
# temp =ox.footprints.osm_footprints_download(polygon=gdf.geometry[0], north=None, south=None, east=None, west=None, footprint_type='building', timeout=180, memory=None, max_query_area_size=2500000000)

In [0]:
temp = ox.core.gdf_from_place('Maranon River', gdf_name='Maranon River', which_result=1, buffer_dist=None)
temp.plot()

In [0]:
tgdf = ox.footprints.footprints_from_polygon(gdf.geometry[0], footprint_type='waterway', retain_invalid=False)
tgdf.plot()

In [0]:
# openfile(False)
shp_path = 'hydrolines.shp'

gdf_water_line = gpd.read_file(shp_path)
gdf_water_line.head()

In [0]:
import geopandas as gpd #
import pandas as pd #
from shapely.geometry import shape
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points
from json import loads
import matplotlib.pyplot as plt


crs = {'init': 'epsg:4326'}
pd.set_option('display.max_columns', None)

In [0]:
gdf_water_line.info()

In [0]:
joined = gpd.sjoin(gdf_water_line, gdf_as,op='intersects')
print(joined.info())
joined

In [0]:
# openfile(False)
shp_path = 'hydropolys.shp'

gdf_water = gpd.read_file(shp_path)
gdf_water.head()